In [1]:
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()
# 2021.10.05 데이터 추출 완료 - 31140 데이터가 대부분 없음, InterfaceError: Python type ResultSet cannot be converted > updowncd 값 2에 의한 에러
cursor = mydb.cursor(prepared=True)
citycode = '35010'

query1 = ("select citycode from public_busctycodelist where citycode >= {}".format(citycode))

cursor.execute(query1)

location_list1 = []

for a in cursor:
    location_list1.append(a)

now = datetime.now()
nowDate = now.strftime('%Y%m%d')

b_cnt = 0
for z in location_list1:
    if b_cnt == 1:
        break
        
    location = z
    query2 = ("select distinct routeid from public_busroute_info where citycode = '{}'".format(location[0]))
    cursor.execute(query2)

    location_list2 = []

    for b in cursor:
        location_list2.append(b)

    cnt = 1
    for i in location_list2:
        routeId = i

        characters = "(',')"

        join_data = ''.join( x for x in routeId if x not in characters)
        print("조회날짜 : {} | location : {} | cnt : {} | start {} public_busroute_info".format(nowDate,location[0],cnt,join_data))

        url = 'http://openapi.tago.go.kr/openapi/service/BusRouteInfoInqireService/getRouteAcctoThrghSttnList'
        queryParams = '?' + urlencode({
            quote_plus('serviceKey') : public_serviceKey(),
            quote_plus('cityCode') : location[0],
            quote_plus('pageNo') : '1',
            quote_plus('numOfRows') : '10000',
            quote_plus('routeId') : join_data
        })

        response = requests.get(url + queryParams).text.encode('utf-8')
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

        rows = xmlobj.findAll('item')
        resultCode = xmlobj.findAll('resultCode')
        if resultCode[0].text == '99':
            print("location {} | {} 번째부터 조회수 제한이 걸렸습니다.".format(location[0],join_data))
            mydb.close()
            b_cnt += 1
            break
        if rows == []:
            print("location {} | {} 데이터가 없습니다.".format(location[0],join_data))
            cnt += 1
            continue
        cnt += 1
        for j in rows:
            item = j
            lat = item.findAll('gpslati')                
            lng = item.findAll('gpslong')
            nodeid = item.findAll('nodeid')
            nodenm = item.findAll('nodenm')
            nodeno = item.findAll('nodeno')
            if nodeno == []:
                nodeno = None
            else:
                nodeno = nodeno[0].text
            nodeord = item.findAll('nodeord')
            routeid = item.findAll('routeid')  
            updowncd = item.findAll('updowncd')
            
            if updowncd == []:
                updowncd = None
            elif updowncd[0].text == '0':
                updowncd = '상행'
            elif updowncd[0].text == '1':
                updowncd = '하행'
            elif updowncd[0].text == '2':
                updowncd = '2'

            sql = "INSERT INTO public_busroute_detail (lat, lng, nodeid, nodenm, nodeno, nodeord, routeid, updowncd, citycode) VALUES (?,?,?,?,?,?,?,?,?)"
            val = (lat[0].text, lng[0].text, nodeid[0].text, nodenm[0].text, nodeno, nodeord[0].text, routeid[0].text, updowncd, location[0])

            cursor.execute(sql, val)
            mydb.commit()

조회날짜 : 20211005 | location : 34330 | cnt : 1 | start TSB294000001 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 2 | start TSB294000002 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 3 | start TSB294000003 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 4 | start TSB294000014 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 5 | start TSB294000015 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 6 | start TSB294000016 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 7 | start TSB294000017 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 8 | start TSB294000018 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 9 | start TSB294000019 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 10 | start TSB294000020 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 11 | start TSB294000021 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt 

조회날짜 : 20211005 | location : 34330 | cnt : 96 | start TSB294000247 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 97 | start TSB294000248 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 98 | start TSB294000249 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 99 | start TSB294000250 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 100 | start TSB294000252 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 101 | start TSB294000254 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 102 | start TSB294000256 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 103 | start TSB294000260 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 104 | start TSB294000261 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 105 | start TSB294000262 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 106 | start TSB294000267 public_busroute_info
조회날짜 : 20211005 | locatio

조회날짜 : 20211005 | location : 34330 | cnt : 190 | start TSB294000340 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 191 | start TSB294000362 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 192 | start TSB294000369 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 193 | start TSB294000370 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 194 | start TSB294000371 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 195 | start TSB294000379 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 196 | start TSB294000036 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 197 | start TSB294000041 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 198 | start TSB294000042 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 199 | start TSB294000043 public_busroute_info
조회날짜 : 20211005 | location : 34330 | cnt : 200 | start TSB294000044 public_busroute_info
조회날짜 : 20211005 | loc

location 34330 | TSB294000030 데이터가 없습니다.
조회날짜 : 20211005 | location : 34330 | cnt : 265 | start TSB294000153 public_busroute_info
location 34330 | TSB294000153 데이터가 없습니다.
조회날짜 : 20211005 | location : 34330 | cnt : 266 | start TSB294000161 public_busroute_info
location 34330 | TSB294000161 데이터가 없습니다.
조회날짜 : 20211005 | location : 34330 | cnt : 267 | start TSB294000162 public_busroute_info
location 34330 | TSB294000162 데이터가 없습니다.
조회날짜 : 20211005 | location : 34330 | cnt : 268 | start TSB294000163 public_busroute_info
location 34330 | TSB294000163 데이터가 없습니다.
조회날짜 : 20211005 | location : 34330 | cnt : 269 | start TSB294000166 public_busroute_info
location 34330 | TSB294000166 데이터가 없습니다.
조회날짜 : 20211005 | location : 34330 | cnt : 270 | start TSB294000189 public_busroute_info
location 34330 | TSB294000189 데이터가 없습니다.
조회날짜 : 20211005 | location : 34330 | cnt : 271 | start TSB294000327 public_busroute_info
location 34330 | TSB294000327 데이터가 없습니다.
조회날짜 : 20211005 | location : 34330 | cnt : 272 | 

location 34390 | TSB300000051 데이터가 없습니다.
조회날짜 : 20211005 | location : 34390 | cnt : 42 | start TSB300000052 public_busroute_info
location 34390 | TSB300000052 데이터가 없습니다.
조회날짜 : 20211005 | location : 34390 | cnt : 43 | start TSB300000056 public_busroute_info
location 34390 | TSB300000056 데이터가 없습니다.
조회날짜 : 20211005 | location : 34390 | cnt : 44 | start TSB300000058 public_busroute_info
location 34390 | TSB300000058 데이터가 없습니다.
조회날짜 : 20211005 | location : 34390 | cnt : 45 | start TSB300000065 public_busroute_info
location 34390 | TSB300000065 데이터가 없습니다.
조회날짜 : 20211005 | location : 34390 | cnt : 46 | start TSB300000066 public_busroute_info
location 34390 | TSB300000066 데이터가 없습니다.
조회날짜 : 20211005 | location : 34390 | cnt : 47 | start TSB300000067 public_busroute_info
location 34390 | TSB300000067 데이터가 없습니다.
조회날짜 : 20211005 | location : 34390 | cnt : 48 | start TSB300000069 public_busroute_info
location 34390 | TSB300000069 데이터가 없습니다.
조회날짜 : 20211005 | location : 34390 | cnt : 49 | start TS

...

location 34390 | TSB300000241 데이터가 없습니다.
조회날짜 : 20211005 | location : 34390 | cnt : 300 | start TSB300000242 public_busroute_info
location 34390 | TSB300000242 데이터가 없습니다.
조회날짜 : 20211005 | location : 34390 | cnt : 301 | start TSB300000248 public_busroute_info
location 34390 | TSB300000248 데이터가 없습니다.
조회날짜 : 20211005 | location : 34390 | cnt : 302 | start TSB300000250 public_busroute_info
location 34390 | TSB300000250 데이터가 없습니다.
조회날짜 : 20211005 | location : 34390 | cnt : 303 | start TSB300000253 public_busroute_info
location 34390 | TSB300000253 데이터가 없습니다.
조회날짜 : 20211005 | location : 34390 | cnt : 304 | start TSB300000254 public_busroute_info
location 34390 | TSB300000254 데이터가 없습니다.
조회날짜 : 20211005 | location : 34390 | cnt : 305 | start TSB300000255 public_busroute_info
location 34390 | TSB300000255 데이터가 없습니다.
조회날짜 : 20211005 | location : 34390 | cnt : 306 | start TSB300000257 public_busroute_info
location 34390 | TSB300000257 데이터가 없습니다.
조회날짜 : 20211005 | location : 34390 | cnt : 307 | 

조회날짜 : 20211005 | location : 35010 | cnt : 33 | start JUB305001140 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 34 | start JUB305001141 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 35 | start JUB305001142 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 36 | start JUB305001143 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 37 | start JUB305001231 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 38 | start JUB305001232 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 39 | start JUB305001233 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 40 | start JUB305001234 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 41 | start JUB305001235 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 42 | start JUB305001236 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 43 | start JUB305001157 public_busroute_info
조회날짜 : 20211005 | location : 350

조회날짜 : 20211005 | location : 35010 | cnt : 127 | start JUB305001534 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 128 | start JUB305001535 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 129 | start JUB305001550 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 130 | start JUB305001551 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 131 | start JUB305001559 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 132 | start JUB305001560 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 133 | start JUB305200003 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 134 | start JUB305200940 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 135 | start JUB305200942 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 136 | start JUB305300257 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 137 | start JUB305000887 public_busroute_info
조회날짜 : 20211005 | loc

조회날짜 : 20211005 | location : 35010 | cnt : 220 | start JUB305001148 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 221 | start JUB305001149 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 222 | start JUB305001150 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 223 | start JUB305001151 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 224 | start JUB305001152 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 225 | start JUB305001153 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 226 | start JUB305001154 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 227 | start JUB305001237 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 228 | start JUB305001238 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 229 | start JUB305001239 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 230 | start JUB305001240 public_busroute_info
조회날짜 : 20211005 | loc

조회날짜 : 20211005 | location : 35010 | cnt : 313 | start JUB305001459 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 314 | start JUB305001460 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 315 | start JUB305001461 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 316 | start JUB305001462 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 317 | start JUB305001463 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 318 | start JUB305001464 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 319 | start JUB305001465 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 320 | start JUB305001482 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 321 | start JUB305001483 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 322 | start JUB305001484 public_busroute_info
조회날짜 : 20211005 | location : 35010 | cnt : 323 | start JUB305001485 public_busroute_info
조회날짜 : 20211005 | loc